In [1]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, SVD, SlopeOne, accuracy
import time
#from sklearn.model_selection import GridSearchCV
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import tabulate as tb

In [2]:
data = Dataset.load_builtin('ml-100k')

In [3]:
sim_options = {
    "n_factors": [x * 50 for x in range(1,5)],
    "n_epochs": [x * 10 for x in range(4, 7)],
    "lr_all": [x / 50 for x in range(1,6)],
    "reg_all": [0.01, 0.02, 0.03]

}
sim_options2 = {
    "n_factors": [50,100],
    "n_epochs": [10,20],
    "lr_all": [0.05],
}
#param_grid = {"sim_options": sim_options}
split = 5
gs = GridSearchCV(SVD, sim_options,
                   measures=['rmse', 'mae'], cv=split, 
                   n_jobs=6, refit=False)
gs.fit(data)

#gs.cv_results_['mean_test_score']
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

0.9345545682914487
{'n_factors': 200, 'n_epochs': 60, 'lr_all': 0.02, 'reg_all': 0.03}


In [ ]:
table = [[] for _ in range(len(gs.cv_results["params"]))]
for i in range(len(gs.cv_results["params"])):
    for key in gs.cv_results.keys():
        table[i].append(gs.cv_results[key][i])

header = gs.cv_results.keys()
print(header)
print(tb.tabulate(table, header, tablefmt="rst"))

print()

for key, val in gs.cv_results.items():
    print("{:<20}".format("'" + key + "':"), end="")
    if isinstance(val[0], float):
        print([float(f"{f:.2f}") for f in val])
    else:
        print(val)

In [5]:
sth = pd.DataFrame(gs.cv_results)
sth.to_csv('result.csv')